# **NLP4_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Variables:
1. Olefin feed (OF) in barrels per day, continuous variable with lower and upper bounds
2. Isobutane recycle (IR) in barrels per day, continuous variable with lower and upper bounds
3. Acid addition rate (AA) in thousands of pounds per day, continuous variable with lower and upper bounds
4. Alkylate yield (AY) in barrels per day, dependent variable 
5. Isobutane makeup (IM) in barrels per day, continuous variable with lower and upper bounds
6. Acid strength (AS), continuous variable with lower and upper bounds
7. Motor Octane Number (MN), dependent variable
8. External isobutane-to-olefin ratio (EO), dependent variable
9. Acid dilution factor (DF), dependent variable
10. F-4 performance number (FN), dependent variable

Objective function to maximize: 
Profit = Revenue - Operational costs
Where,
Revenue = AY*MN
Operational costs = OF + IR + AA + IM

Relationship between variables:
1. AY = OF *(1.12 + 0.13167*EO -0.00667*EO^2) ± 2%
2. MN = 86.35 + 1.098*EO - 0.038*EO^2 + 0.325

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Maximize P = AY*MN - (OF + IR + AA + IM)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. AY = OF *(1.12 + 0.13167*EO - 0.00667*EO^2) ± 2%
2. MN = 86.35 + 1.098*EO - 0.038*EO^2 + 0.325*(AS - 89) ± 1%
3. DF = 35.82 - 0.222*FN ± 5%
4. FN = -133 + 3*MN ± 10%
5. EO = (IR + IM) / OF
6. AS = 93000* AA / (AA *DF + 1000*AA)
7. 1.22 AY = OF + IM
8. Lower and upper bound constraints for all variables.


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Below is the Python code using Pyomo:

```python
from pyomo.environ import *

model = ConcreteModel()

# Define bounds for decision variables
OF_bounds = (100, 200)  # sample bounds for Olefin feed 
IR_bounds = (50, 100)   # sample bounds for Isobutane recycle 
AA_bounds = (10, 20)    # sample bounds for Acid addition rate
IM_bounds = (50, 100)   # sample bounds for Isobutane makeup 
AS_bounds = (88, 90)    # sample bounds for Acid strength 

model.OF = Var(bounds=OF_bounds)
model.IR = Var(bounds=IR_bounds)
model.AA = Var(bounds=AA_bounds)
model.IM = Var(bounds=IM_bounds)
model.AS = Var(bounds=AS_bounds)
model.AY = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.MN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.EO = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.DF = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.FN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable

# Define Objective
model.profit = Objective(expr=model.AY*model.MN - (model.OF + model.IR + model.AA + model.IM), sense=maximize)

# Define Constraints
model.const1 = Constraint(expr=model.AY == model.OF *(1.12 + 0.13167*model.EO -0.00667*model.EO**2))
model.const2 = Constraint(expr=model.MN == 86.35 + 1.098*model.EO - 0.038*model.EO**2 + 0.325*(model.AS - 89))
model.const3 = Constraint(expr=model.DF == 35.82 - 0.222*model.FN)
model.const4 = Constraint(expr=model.FN == -133 + 3*model.MN)
model.const5 = Constraint(expr=model.EO == (model.IR + model.IM) / model.OF)
model.const6 = Constraint(expr=model.AS == 93000* model.AA / (model.AA *model.DF + 1000*model.AA))
model.const7 = Constraint(expr=1.22 * model.AY == model.OF + model.IM)

SolverFactory('glpk').solve(model)

model.pprint()

print('\nProfit = ', model.profit())

```

Please note that the provided objective function and constraints lack a stochastic element in the equations which could absorb the expressed variability (± percentage). Normally, an optimization model would incorporate variability within the constraints by formulating stochastic programming or robust optimization models.

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.5 MB/s eta 0:00:00


In [14]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [15]:
from pyomo.environ import *

model = ConcreteModel()

# Define bounds for decision variables
OF_bounds = (100, 200)  # sample bounds for Olefin feed
IR_bounds = (50, 100)   # sample bounds for Isobutane recycle
AA_bounds = (10, 20)    # sample bounds for Acid addition rate
IM_bounds = (50, 100)   # sample bounds for Isobutane makeup
AS_bounds = (88, 90)    # sample bounds for Acid strength

model.OF = Var(bounds=OF_bounds)
model.IR = Var(bounds=IR_bounds)
model.AA = Var(bounds=AA_bounds)
model.IM = Var(bounds=IM_bounds)
model.AS = Var(bounds=AS_bounds)
model.AY = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.MN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.EO = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.DF = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.FN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable

# Define Objective
model.profit = Objective(expr=model.AY*model.MN - (model.OF + model.IR + model.AA + model.IM), sense=maximize)

# Define Constraints
model.const1 = Constraint(expr=model.AY == model.OF *(1.12 + 0.13167*model.EO -0.00667*model.EO**2))
model.const2 = Constraint(expr=model.MN == 86.35 + 1.098*model.EO - 0.038*model.EO**2 + 0.325*(model.AS - 89))
model.const3 = Constraint(expr=model.DF == 35.82 - 0.222*model.FN)
model.const4 = Constraint(expr=model.FN == -133 + 3*model.MN)
model.const5 = Constraint(expr=model.EO == (model.IR + model.IM) / model.OF)
model.const6 = Constraint(expr=model.AS == 93000* model.AA / (model.AA *model.DF + 1000*model.AA))
model.const7 = Constraint(expr=1.22 * model.AY == model.OF + model.IM)

SolverFactory('couenne').solve(model)

model.pprint()

print('\nProfit = ', model.profit())

  - termination condition: infeasible
  - message from solver: couenne\x3a Infeasible problem
ERROR:pyomo.common.numeric_types:evaluating object as numeric value: AY
    (object: <class 'pyomo.core.base.var.ScalarVar'>)
No value for uninitialized NumericValue object AY


10 Var Declarations
    AA : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    10 :  None :    20 : False :  True :  Reals
    AS : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    88 :  None :    90 : False :  True :  Reals
    AY : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    DF : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    EO : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    FN : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    IM : Size=1, Index=None


ValueError: No value for uninitialized NumericValue object AY

In [24]:
from pyomo.environ import *

model = ConcreteModel()

# Define bounds for decision variables
OF_bounds = (0, 2000)  # sample bounds for Olefin feed
IR_bounds = (0, 16000)   # sample bounds for Isobutane recycle
AA_bounds = (0, 120)    # sample bounds for Acid addition rate
IM_bounds = (0, 2000)   # sample bounds for Isobutane makeup
AS_bounds = (85, 93)    # sample bounds for Acid strength
AY_bounds = (0,5000)
MN_bounds = (90,95)
EO_bounds = (3,12)
DF_bounds = (12,4)
FN_bounds = (145,162)


model.OF = Var(bounds=OF_bounds)
model.IR = Var(bounds=IR_bounds)
model.AA = Var(bounds=AA_bounds)
model.IM = Var(bounds=IM_bounds)
model.AS = Var(bounds=AS_bounds)
model.AY = Var(within=NonNegativeReals, bounds=AY_bounds)  # Taken as continuous, nonnegative variable
model.MN = Var(within=NonNegativeReals, bounds=MN_bounds)  # Taken as continuous, nonnegative variable
model.EO = Var(within=NonNegativeReals, bounds=EO_bounds)  # Taken as continuous, nonnegative variable
model.DF = Var(within=NonNegativeReals, bounds=DF_bounds)  # Taken as continuous, nonnegative variable
model.FN = Var(within=NonNegativeReals, bounds=FN_bounds)  # Taken as continuous, nonnegative variable

# Define Objective
model.profit = Objective(expr=model.AY*model.MN - (model.OF + model.IR + model.AA + model.IM), sense=maximize)

# Define Constraints
model.const1 = Constraint(expr=model.AY == model.OF *(1.12 + 0.13167*model.EO -0.00667*model.EO**2))
model.const2 = Constraint(expr=model.MN == 86.35 + 1.098*model.EO - 0.038*model.EO**2 + 0.325*(model.AS - 89))
model.const3 = Constraint(expr=model.DF == 35.82 - 0.222*model.FN)
model.const4 = Constraint(expr=model.FN == -133 + 3*model.MN)
model.const5 = Constraint(expr=model.EO == (model.IR + model.IM) / model.OF)
model.const6 = Constraint(expr=model.AS == 93000* model.AA / (model.AA *model.DF + 1000*model.AA))
model.const7 = Constraint(expr=1.22 * model.AY == model.OF + model.IM)

SolverFactory('couenne').solve(model)

model.pprint()
print(model.profit())


  - termination condition: infeasible
  - message from solver: Couenne (/tmp/tmpagm606gu.pyomo.nl Feb 21 2023)\x3a Infeasible
ERROR:pyomo.common.numeric_types:evaluating object as numeric value: AY
    (object: <class 'pyomo.core.base.var.ScalarVar'>)
No value for uninitialized NumericValue object AY


10 Var Declarations
    AA : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :   120 : False :  True :  Reals
    AS : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    85 :  None :    93 : False :  True :  Reals
    AY : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  5000 : False :  True : NonNegativeReals
    DF : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    12 :  None :     4 : False :  True : NonNegativeReals
    EO : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     3 :  None :    12 : False :  True : NonNegativeReals
    FN : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   145 :  None :   162 : False :  True : NonNegativeReals
    IM : Size=1, Index=None


ValueError: No value for uninitialized NumericValue object AY

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [23]:
from pyomo.environ import *

model = ConcreteModel()


model.OF = Var(within=NonNegativeReals)
model.IR = Var(within=NonNegativeReals)
model.AA = Var(within=NonNegativeReals)
model.IM = Var(within=NonNegativeReals)
model.AS = Var(within=NonNegativeReals)
model.AY = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.MN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.EO = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.DF = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable
model.FN = Var(within=NonNegativeReals)  # Taken as continuous, nonnegative variable

# Define Objective
model.profit = Objective(expr=model.AY*model.MN - (model.OF + model.IR + model.AA + model.IM), sense=maximize)

# Define Constraints
model.const1 = Constraint(expr=model.AY >= model.OF *(1.12 + 0.13167*model.EO -0.00667*model.EO**2) - 0.02*model.AY)
model.const11 = Constraint(expr=model.AY <= model.OF *(1.12 + 0.13167*model.EO + 0.00667*model.EO**2) + 0.02*model.AY)


model.const2 = Constraint(expr=model.MN >= 86.35 + 1.098*model.EO - 0.038*model.EO**2 + 0.325*(model.AS - 89) - 0.01*model.MN)
model.const22 = Constraint(expr=model.MN <= 86.35 + 1.098*model.EO - 0.038*model.EO**2 + 0.325*(model.AS - 89) + 0.01*model.MN)

model.const3 = Constraint(expr=model.DF >= 35.82 - 0.222*model.FN - 0.05*model.DF)
model.const33 = Constraint(expr=model.DF <= 35.82 - 0.222*model.FN + 0.05*model.DF)

model.const4 = Constraint(expr=model.FN >= -133 + 3*model.MN - 0.1*model.FN)
model.const44 = Constraint(expr=model.FN <= -133 + 3*model.MN + 0.1*model.FN)

model.const5 = Constraint(expr=model.EO == (model.IR + model.IM) / model.OF)
model.const6 = Constraint(expr=model.AS == 93000* model.AA / (model.AA *model.DF + 1000*model.AA))
model.const7 = Constraint(expr=1.22 * model.AY == model.OF + model.IM)

model.OF.setub(2000)
model.OF.setlb(0)
model.IR.setub(16000)
model.IR.setlb(0)
model.AA.setub(120)
model.AA.setlb(0)
model.AY.setub(5000)
model.AY.setlb(0)
model.IM.setub(2000)
model.IM.setlb(0)
model.AS.setub(93)
model.AS.setlb(85)
model.MN.setub(95)
model.MN.setlb(90)
model.EO.setub(12)
model.EO.setlb(3)
model.DF.setub(4)
model.DF.setlb(1.2)
model.FN.setub(162)
model.FN.setlb(145)

SolverFactory('couenne').solve(model)

model.pprint()
print(model.profit())


  - termination condition: infeasible
  - message from solver: couenne\x3a Infeasible problem


10 Var Declarations
    AA : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :   120 : False : False : NonNegativeReals
    AS : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :    85 : 92.99999999999997 :    93 : False : False : NonNegativeReals
    AY : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 3278.6885245901644 :  5000 : False : False : NonNegativeReals
    DF : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   1.2 :   1.2 :     4 : False : False : NonNegativeReals
    EO : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     3 : 6.043362996704711 :    12 : False : False : NonNegativeReals
    FN : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : D